In [18]:
#import sklearn
#import tensorflow as tf
#import keras

import pandas as pd
import numpy as np
#import scipy
import torch

In [ ]:
#from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample = pd.read_csv("data/sample_submission.csv")


In [3]:
train.head(5)

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [46]:
train.drop("id",inplace=True,axis=1)
test.drop("id",inplace=True,axis=1)
id = sample["id"]
names = sample.columns.tolist()
sample.drop("id",inplace=True,axis=1)


KeyError: "['id'] not found in axis"

In [5]:
y_col = sample.columns.tolist()

y_train = train[y_col]
X_train = train.drop(y_col,axis=1)

print(X_train.shape)
print(y_train.shape)

(19219, 27)
(19219, 7)


In [6]:
X_train.head(10)

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,584,590,909972,909977,16,8,5,2274,113,140,...,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998
5,1009,1033,899231,899307,409,22,26,47513,86,118,...,0.0053,0.5454,1.0000,1.0,2.6117,0.9542,1.4150,0.5909,-0.1890,0.8749
6,596,607,739072,7390760,204,21,21,22478,89,127,...,0.0110,0.7143,1.0000,1.0,2.3096,1.1761,1.3222,0.3158,-0.1497,0.5212
7,1673,1687,294065,294091,571,38,57,53142,77,110,...,0.0142,0.5686,0.7179,1.0,2.7528,1.3802,1.7559,0.0357,-0.2661,0.9408
8,507,521,203252,203261,101,25,12,12530,124,140,...,0.0176,0.8000,0.7500,0.0,2.0043,1.3802,0.7782,-0.6667,0.0305,0.3601
9,893,907,1341292,1341296,54,6,15,7052,87,133,...,0.0037,0.8333,1.0000,1.0,1.7324,0.6990,1.1761,0.4667,-0.1228,0.1400


In [7]:
x = torch.Tensor(X_train.values)
y = torch.Tensor(y_train.values)

In [33]:
device = torch.device('cpu')
N, D_in,  D_out = 19219, 27, 7
H = [500,1000,500,100]

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H[0]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[0], H[1]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[1], H[2]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[2], H[3]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[3], D_out),
    
)

l_r = 1e-9
for i in range(100):
    y_pred = model(x)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred,y)
    print(loss)
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= l_r*param.grad
    model.zero_grad()

tensor(1019.2882, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1008.9661, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(998.6417, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(988.3156, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(977.9898, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(967.6619, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(957.3314, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(946.9979, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(936.6613, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(926.3218, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(915.9821, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(905.6456, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(895.3141, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(884.9859, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(874.6595, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(8

In [14]:
y1 = y[:,1].reshape(19219, 1)

In [31]:
device = torch.device('cpu')
N, D_in,  D_out = 19219, 27, 7
H = [500,1000,500,100]

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H[0]),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H[0], H[1]),
    torch.nn.ReLU(),
    torch.nn.Linear(H[1], H[2]),
    torch.nn.ReLU(),
    torch.nn.Linear(H[2], D_out)#,
 #   torch.nn.Softmax()
    
)

l_r = 1e-2
for i in range(200):
    l_r  = 1e-9/(i+1)
    y_pred = model(x)
    #loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred,y1)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(y_pred,y)
    print(loss)
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= l_r*param.grad
    model.zero_grad()

tensor(8156.2090, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7996.6626, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7916.9038, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7863.7451, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7823.8882, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7792.0083, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7765.4438, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7742.6772, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7722.7583, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7705.0532, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7689.1206, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7674.6377, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7661.3623, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7649.1089, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(7637.7324, grad_fn=<BinaryCrossEntropyWithLogitsBackwar

In [60]:
X_test_2l = torch.Tensor(test.values)
y_pred_2l = model(X_test_2l)
#torch.nn.functional.softmax(y_pred_2l)
y_pred_2l = pd.DataFrame(y_pred_2l.detach()).astype("float")
y_pred_2l

,0,1,2,3,4,5,6
0,-9310.695312,-698.863647,-3670.098877,-1596.719360,-9363.209961,-34.862896,-17029.802734
1,-1010.455322,4.691124,-357.533600,-266.570190,-917.515381,116.367203,-1896.288940
2,-328.772766,88.712715,-111.429024,-88.732803,-249.909714,83.305862,-554.462097
3,-555.294434,-26.209236,-220.011230,-119.392715,-559.635742,10.198504,-1042.587402
4,-1311.593994,-77.325645,-513.678528,-255.395767,-1316.700806,14.029071,-2431.389160
...,...,...,...,...,...,...,...
12809,-1065.787354,-37.583691,-409.704041,-243.751358,-1056.371460,41.240864,-2008.474731
12810,-7785.365723,-737.383362,-3114.756104,-1099.657837,-7822.639648,-175.881454,-14005.721680
12811,-11686.697266,3057.932129,-6981.744629,-427.388550,-5714.283691,1534.834106,-3238.153320
12812,-1667.892212,-53.381248,-638.866272,-390.085999,-1643.164673,73.879776,-3148.392822


In [53]:
y_pred_2l = pd.DataFrame(y_pred_2l.detach()).astype("float")


In [54]:
from sklearn.preprocessing import quantile_transform
out_prob= quantile_transform(y_pred_2l)
out_prob = pd.DataFrame(out_prob)
out_prob

,0,1,2,3,4,5,6
0,0.149994,0.136169,0.159744,0.152661,0.110415,0.363860,0.112813
1,0.851671,0.738112,0.862576,0.759227,0.845507,0.740958,0.807648
2,0.944442,0.800568,0.953188,0.901234,0.939881,0.716384,0.909873
3,0.908666,0.678707,0.908315,0.876137,0.889081,0.523087,0.867507
4,0.812023,0.605388,0.811529,0.770819,0.792756,0.542310,0.767363
...,...,...,...,...,...,...,...
12809,0.842156,0.660195,0.842339,0.781346,0.823586,0.645136,0.796988
12810,0.193460,0.127564,0.192611,0.253261,0.138762,0.093238,0.138257
12811,0.071502,0.963727,0.035715,0.611738,0.243976,0.934656,0.699697
12812,0.763713,0.636348,0.764448,0.643643,0.743231,0.705894,0.709681


In [56]:


out_prob.columns = sample.columns
out_prob["id"] = id

In [61]:
out_prob = out_prob[names]
out_prob.to_csv("output/2lnn_3.csv",index=False)